In [1]:
import os
from src.clients.bluesky_client import BlueskyClient
from src.core.config import settings

# Configura a chave de API do Google para o LangChain
os.environ['GOOGLE_API_KEY'] = settings.GOOGLE_API_KEY

print("Ambiente configurado. Cliente Bluesky pronto para uso.")

Ambiente configurado. Cliente Bluesky pronto para uso.


In [2]:
# Célula 2 (VERSÃO ATUALIZADA)

# Instancia e faz o login no Bluesky
bsky = BlueskyClient()
bsky.login()

# Vamos buscar posts usando uma palavra-chave comum para garantir que teremos resultados.
# Tente usar termos genéricos como "Brasil", "tecnologia", "IA", etc.
search_query = "Charlie Kirk"
posts = bsky.search_posts(query=search_query, limit=1000)

# Extrai e limpa o texto dos posts
# A estrutura do objeto de post da busca é um pouco diferente, por isso ajustamos o código
post_texts = [p.record.text for p in posts if hasattr(p, 'record') and hasattr(p.record, 'text')]

print(f"Coletados {len(post_texts)} posts para a busca '{search_query}'.")
if post_texts:
    print("Amostra do primeiro post:", post_texts[0][:150] + "...")

Login bem-sucedido como: 
Coletados 1000 posts para a busca 'Charlie Kirk'.
Amostra do primeiro post: Donald Trump is preaching lies to his MAGA Base which are Christian White Nationalists, for whom Charlie Kirk is 

A Christian martyr who died for 
hi...


In [3]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

print("Iniciando o processo de embedding. Isso pode levar um momento...")

# Inicializa o modelo de embedding. Ele será baixado na primeira vez.
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Cria o banco de dados vetorial em memória com os textos dos posts
vector_store = FAISS.from_texts(texts=post_texts, embedding=embeddings)

print("Base de conhecimento vetorial criada com sucesso!")

Iniciando o processo de embedding. Isso pode levar um momento...


C:\Users\joaoo\AppData\Local\Temp\ipykernel_33892\2297149645.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
C:\Users\joaoo\AppData\Local\pypoetry\Cache\virtualenvs\ask-the-sky-Gtuqa6oy-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Base de conhecimento vetorial criada com sucesso!


In [4]:
# Célula 4 (VERSÃO ATUALIZADA para GPT-4o mini)
import os
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from src.core.config import settings

# Configura a chave da OpenAI para o LangChain
os.environ['OPENAI_API_KEY'] = settings.OPENAI_API_KEY

# Inicializa o LLM (GPT-4o mini)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# Cria a cadeia de RAG (esta parte não muda!)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 5}) # Busca os 5 chunks mais relevantes
)

# Agora, fazemos a pergunta!
question = "Quais as ultimas novidades sobre o crime? Ja se sabe se o atirador agiu sozinho?"

print(f"\nPergunta: {question}")

# Invoca a cadeia para obter a resposta
response = rag_chain.invoke(question)

print("\n--- Resposta da IA (GPT-4o mini) ---")
print(response['result'])


Pergunta: Quais as ultimas novidades sobre o crime?

--- Resposta da IA (GPT-4o mini) ---
Não sei.
